In [70]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import cv2
import os
from tensorflow import keras

from keras.layers import (GlobalAveragePooling2D, Conv2D, MaxPool2D, Dense,
                          Flatten, InputLayer, BatchNormalization, Input,
                          Dropout,Bidirectional, LSTM ,GRU, MaxPooling2D , Reshape, RandomRotation,RandomContrast,RandomBrightness)

from keras import backend as k
from keras.optimizers import Adam , SGD 
from keras.callbacks import  ReduceLROnPlateau 
from keras.regularizers  import L2
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Lambda

from keras import layers
from keras.preprocessing.image import img_to_array, load_img


In [2]:
from itertools import groupby
char_list =['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
def ctc_decoder(predictions):
    '''
    input: given batch of predictions from text rec model
    output: return lists of raw extracted text

    '''
    text_list = []
    
    pred_indcies = np.argmax(predictions, axis=-1)
    
    for i in range(pred_indcies.shape[0]):
        ans = ""
        
        ## merge repeats
        merged_list = [k for k,_ in groupby(pred_indcies[i])]
        
        ## remove blanks
        for p in merged_list:
            if p != len(char_list):
                ans += char_list[int(p)]
        
        text_list.append(ans)
        
    return text_list

In [ ]:
!pip install ultralytics

In [4]:
from ultralytics import YOLO
model1 = YOLO("/kaggle/input/models2/model/yolov8m-pose.pt" , verbose=False)

In [8]:
def extract_jersey_from_frame(img_path):
    image = cv2.imread(img_path)
    if image is None :
        return None
    im_height, im_width = image.shape[:2]
    if im_height<75 or im_width<35:
        return None

    image = cv2.resize(image , (285,600))
    im_height, im_width = image.shape[:2]
    results = model1(image)
    keypoints = results[0].keypoints
    threshold = 0.1
    points = []

    for kpt_data in keypoints.data[0]:
        # Extract the confidence (probability) of the keypoint
        prob = kpt_data[2].item()

        # If the confidence is above the threshold, consider it as a valid keypoint
        if prob > threshold:
            x = kpt_data[0].item()
            y = kpt_data[1].item()
            points.append((int(x), int(y)))
        else:
            points.append(None)
    if  keypoints.has_visible is False:
        points = [None] * 13
    quadrilateral_coords = [points[6], points[5], points[12], points[11]]
    
    # Check if any of the points are None
    if None in quadrilateral_coords:
        return None
#         print(f"Error: Not enough valid points detected for image at index {j}. Skipping...")
        
    x1, y1 = points[5]
    x2, y2 = points[6]
    x3, y3 = points[11]
    x4, y4 = points[12]
    top = np.abs(min(y1,y2,y3,y4))
    left = np.abs(min(x1,x2,x3,x4))
    height = np.abs(max(y1,y2,y3,y4)-top)
    width = np.abs(max(x1,x2,x3,x4)-left)
    if top >= im_height * 0.5 or left >= im_width*0.7:
        return None
    if 0 in [top, height, width, left]:
        return None
    img1 = image[int(top+height*.1-3):int(top+height*.8+3), int(left-10):int(left+width+10)]
    try:
        img1 = cv2.resize(img1 , (64,32))
    except:
        return None
    if x1 >= x2 or x1 >= x4 or x3 >= x4 or x3 >= x2 or  y1 >= y3 or y1 >=y4 or y2 >= y3 or y2 >= y4 or np.abs(x2 - x1) < 55 or np.abs(x1 - x2) < 55 :
        return None
    else:
        return img1

In [9]:
class CTCLayer(layers.Layer):

    def __init__(self, name=None):

        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.

        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

In [10]:
from itertools import groupby
char_list =['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
def ctc_decoder(predictions):
    '''
    input: given batch of predictions from text rec model
    output: return lists of raw extracted text

    '''
    text_list = []
    
    pred_indcies = np.argmax(predictions, axis=-1)
    
    for i in range(pred_indcies.shape[0]):
        ans = ""
        
        ## merge repeats
        merged_list = [k for k,_ in groupby(pred_indcies[i])]
        
        ## remove blanks
        for p in merged_list:
            if p != len(char_list):
                ans += char_list[int(p)]
        
        text_list.append(ans)
        
    return text_list



In [11]:
inputs = Input(shape=(32, 64, 1) , name = "image")
labels = layers.Input(name="label", shape=(None,), dtype="float32")

x = Conv2D(32, (4,4), padding='same', activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 1))(x)

x = Conv2D(64, (2, 3), padding='same', activation='relu')(x)
# x = Dropout(0.3)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (2, 3), padding='same', activation='relu')(x)
x = Dropout(0.22)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 1))(x)


x = Conv2D(256, (2,3),padding='same', activation='relu')(x)
x = Dropout(0.22)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 1))(x)

conv_6 = Conv2D(32, (2,2), activation = "relu")(x)
x = Dropout(0.3)(x)

squeezed = keras.layers.Lambda(lambda x: keras.backend.squeeze(x, 1) , output_shape=(31, 32))(conv_6)

# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(GRU(256, return_sequences=True , dropout=0.25))(squeezed)
blstm_2 = Bidirectional(GRU(128, return_sequences=True , dropout=0.25))(blstm_1)

softmax_output = Dense(len(char_list)+1, activation = 'softmax', name="dense")(blstm_2)

output = CTCLayer(name="ctc_loss")(labels, softmax_output)
optimizer = Adam(learning_rate=0.001 ,beta_1=0.11, beta_2=0.99, clipnorm=1.0)

#model to be used at training time
model = keras.models.Model(inputs=[inputs, labels], outputs=output)
model.compile(optimizer = optimizer)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 32, 64, 1) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 64,    │        544 │ image[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 64,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 16, 64,    │     12,352 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 8, 32, 64) │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 8, 32,     │     49,280 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 8, 32,     │          0 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 8, 32,     │        512 │ dropout[0][0]     │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 4, 32,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 4, 32,     │    196,864 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 4, 32,     │          0 │ conv2d_3[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 32,     │      1,024 │ dropout_1[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 2, 32,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 1, 31, 32) │     32,800 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 31, 32)    │          0 │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 31, 512)   │    445,440 │ lambda[0][0]      │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 31, 256)   │    493,056 │ bidirectional[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ label (InputLayer)  │ (None, None)      │          0 │ -               

 Total params: 1,234,699 (4.71 MB)

 Trainable params: 1,233,931 (4.71 MB)

 Non-trainable params: 768 (3.00 KB)

In [12]:
model.load_weights('/kaggle/input/jersey-best-models/my_checkpoint90.weights.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 58 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [13]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 32, 64, 1) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 64,    │        544 │ image[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 64,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 16, 64,    │     12,352 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 8, 32, 64) │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 8, 32,     │     49,280 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 8, 32,     │          0 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 8, 32,     │        512 │ dropout[0][0]     │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 4, 32,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 4, 32,     │    196,864 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 4, 32,     │          0 │ conv2d_3[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 32,     │      1,024 │ dropout_1[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 2, 32,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 1, 31, 32) │     32,800 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 31, 32)    │          0 │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 31, 512)   │    445,440 │ lambda[0][0]      │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 31, 256)   │    493,056 │ bidirectional[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ label (InputLayer)  │ (None, None)      │          0 │ -               

 Total params: 1,234,699 (4.71 MB)

 Trainable params: 1,233,931 (4.71 MB)

 Non-trainable params: 768 (3.00 KB)

In [15]:
prediction_model = keras.models.Model(
    model.input[0],
    model.get_layer(name="dense").output
)

In [16]:
new_image_path = "/kaggle/input/number-for-testing/9.jpg"
new_image = cv2.imread(new_image_path)
new_image = cv2.resize(new_image, (64, 32))  
new_image = cv2.cvtColor(new_image ,  cv2.COLOR_BGR2GRAY)
new_image = np.expand_dims(new_image, axis=0)  

In [17]:
preds = prediction_model.predict(new_image)
pred_texts = ctc_decoder(preds)
print(pred_texts)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 991ms/step
['9']


In [52]:
from collections import Counter

def calculate_percentage(lst):
    counts = Counter(lst)
    total_count = len(lst)
    percentages = {key: (count / total_count) * 100 for key, count in counts.items()}
    sorted_percentages = dict(sorted(percentages.items(), key=lambda item: item[1], reverse=True))
    return sorted_percentages
# Example usage:

percentage = calculate_percentage(label)
print("Percentage of each element:", percentage)


Percentage of each element: {}


In [53]:
def get_highest_percentages(percentage_dict):
    # Sort the dictionary by values in descending order
    sorted_percentage = sorted(percentage_dict.items(), key=lambda x: x[1], reverse=True)
    
    # Get the first two elements
    highest_percentages = sorted_percentage[:2]
    
    # Extract keys and values from the first two elements
    keys = [item[0] for item in highest_percentages]
    values = [item[1] for item in highest_percentages]
    
    return keys, values

# Example dictionary
# percentage = {'8': 10, '-1': 20, '6': 30, '18': 40, '75': 50, '0': 60, '1': 70}

# Call the function
keys, values = get_highest_percentages(percentage)

print("Keys of first two highest percentages:", keys)
print("Values of first two highest percentages:", values)


Keys of first two highest percentages: []
Values of first two highest percentages: []


In [76]:
from collections import Counter

def predict_majority_jersey_no_for_playerid(player_id, pred_label_list):
    """
    Predicts the majority jersey number for a player based on a list of labels.

    Args:
        player_id (int): The id number associated with the player.
        pred_label_list (list): A list of class labels.

    Returns:
        dict: A dictionary containing the player_id and the predicted label.
    """
    # Count occurrences of each label
    counts = Counter(pred_label_list)
    total_count = len(pred_label_list)
    
    # Calculate the percentage of each label
    percentages = {key: (count / total_count) * 100 for key, count in counts.items()}
    
    # Sort labels by percentage in descending order
    sorted_percentage = sorted(percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Get the top two elements with the highest percentage
    top_two = sorted_percentage[:2]
    
    # Extract keys and values from the top two elements
    keys = [item[0] for item in top_two]
    values = [item[1] for item in top_two]
    if len(keys)!= 0:
    # Check conditions to determine the predicted label
        if values[0] > 98.5 and keys[0] == "-1":
            pred_label = int(keys[0])  # If "-1" has a very high percentage, choose it
        elif int(values[0]) > int(values[1]) and keys[0] != "-1":
            pred_label = int(keys[0])  # Choose the label with the highest percentage if it's not "-1"
        else:
            pred_label = int(keys[1])  # Otherwise, choose the second label
    else:
        pred_label= -1
    # Create a dictionary with the predicted label for the player
    pred_dict = {str(player_id): pred_label}
    
    return pred_dict


In [ ]:
predictions = {}

# Loop through player IDs
for player_id in range(1426):
    pred_label_list = []
    
    # Define the path to the folder containing player frames
    player_frame_folder_path = os.path.join("/kaggle/input/soccernet/challenge/images/", str(player_id))
    
    # Get paths to each frame for the player
    player_frames_path = [os.path.join(player_frame_folder_path, frame) for frame in os.listdir(player_frame_folder_path)]
    
    # Process each frame
    for frame_path in player_frames_path:
        print(frame_path)
        
        # Extract jersey from the frame
        jersey_extract_roi = extract_jersey_from_frame(frame_path)
        
        if jersey_extract_roi is not None:
            # Resize and convert the extracted jersey to grayscale
            resized_jersey_extract_roi = cv2.resize(jersey_extract_roi, (64,32))  
            gray_jersey_extract_roi = cv2.cvtColor(resized_jersey_extract_roi, cv2.COLOR_BGR2GRAY)
            gray_jersey_extract_roi = new_image = np.expand_dims(gray_jersey_extract_roi, axis=0)  
            # Use EasyOCR to recognize text (jersey numbers) from the grayscale image
            preds = prediction_model.predict(gray_jersey_extract_roi)
            pred_texts = ctc_decoder(preds)
            print(pred_texts)
            if pred_texts==['']:

                pred_label_list.append("-1")
            else:
                pred_label_list.append(pred_texts[0])
        else:
            pred_label_list.append("-1")
    # Predict the majority jersey number for the player
    predictions_dic = predict_majority_jersey_no_for_playerid(player_id, pred_label_list)
    
    # Add the prediction to the overall predictions dictionary
    predictions = {**predictions, **predictions_dic}

/kaggle/input/soccernet/Data/SoccerNet/jersey-2023/challenge/challenge/images/8/8_248.jpg

0: 640x320 1 person, 469.9ms
Speed: 2.6ms preprocess, 469.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 320)
/kaggle/input/soccernet/Data/SoccerNet/jersey-2023/challenge/challenge/images/8/8_210.jpg

0: 640x320 1 person, 453.7ms
Speed: 2.3ms preprocess, 453.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 320)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
['']
/kaggle/input/soccernet/Data/SoccerNet/jersey-2023/challenge/challenge/images/8/8_468.jpg

0: 640x320 1 person, 458.0ms
Speed: 2.5ms preprocess, 458.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 320)
/kaggle/input/soccernet/Data/SoccerNet/jersey-2023/challenge/challenge/images/8/8_209.jpg

0: 640x320 2 persons, 448.5ms
Speed: 2.4ms preprocess, 448.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 320)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
['']
/kaggle/input/soccernet/Data/SoccerNet/je

In [ ]:
predictions

In [ ]:
import json


# File path where you want to save the JSON file
file_path = 'output.json'

# Save dictionary to JSON file
with open(file_path, 'w') as json_file:
    json.dump(predictions, json_file)

print(f"JSON data saved to {file_path}")

